In [1]:
# Kütüphaneleri import etme
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from sklearn.model_selection import train_test_split
import shutil

In [2]:
data_dir = 'C:/Users/esraa/Desktop/ham10000'
# Change the current working directory
os.chdir(data_dir)

# Görüntüler için yeni bir klasör oluşturma
base_dir = data_dir+'/base_dir'
os.mkdir(base_dir)

# Eğitim dosyası klasörü oluşturma
train_dir = os.path.join(base_dir, 'train_dir')
os.mkdir(train_dir)

# Doğrulama dosyası klasörü oluşturma
val_dir = os.path.join(base_dir, 'val_dir')
os.mkdir(val_dir)


In [3]:
# Eğitim ve doğrulama sınıfları için yeni klasörler oluşturma
class_labels = ['akiec', 'bcc', 'bkl', 'df', 'mel','nv', 'vasc']
for folder_names in [train_dir, val_dir]:
   for name in class_labels:
      os.mkdir(os.path.join(folder_names, name))

In [4]:
# metadata 'yı okuma
df = pd.read_csv('HAM10000_metadata.csv')

# # Veri setindeki bazı bilgileri görüntüleme
df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [5]:
# Etiket kolonunu y olarak alma
y = df['dx']

In [6]:
# metadata 'yı eğitim ve doğrulama kümelerine ayırma
df_train, df_val = train_test_split(df,
                                    test_size=0.1,
                                    random_state=101,
                                    stratify=y)

In [7]:
# Eğitim ve doğrulamanın şeklini yazdırma
print(df_train.shape)
print(df_val.shape)


(9013, 7)
(1002, 7)


In [8]:
# # Eğitim ve doğrulama kümelerindeki değerlerin sayısını bulma
df_train['dx'].value_counts()

dx
nv       6034
mel      1002
bkl       989
bcc       463
akiec     294
vasc      128
df        103
Name: count, dtype: int64

In [9]:
df_val['dx'].value_counts()

dx
nv       671
mel      111
bkl      110
bcc       51
akiec     33
vasc      14
df        12
Name: count, dtype: int64

In [10]:
# Görüntüleri klasörlere transfer etme
# Görüntü id'sini indeks olarak ayarlama
df.set_index('image_id', inplace=True)

In [11]:
# Klasörlerdeki her birindeki görüntülerin listesini alma
folder_1 = os.listdir(data_dir + '/ham10000_images_part_1')
folder_2 = os.listdir(data_dir + '/ham10000_images_part_2')

# Eğitim ve doğrulama görüntülerinin id listesini alma
train_list = list(df_train['image_id'])
val_list = list(df_val['image_id'])

In [12]:
# Eğitim görüntülerini transfer etme
# Eğitim görüntülerini trasfer etme
for image in train_list:

    fname = image + '.jpg'
    label = df.loc[image, 'dx']

    # Dosyanın hangi klasörde olduğunu kontrol etme
    if fname in folder_1:
        src = os.path.join(data_dir, 'ham10000_images_part_1', fname)
    elif fname in folder_2:

        src = os.path.join(data_dir, 'ham10000_images_part_2', fname)
    else:
        continue  # Dosya bulunamazsa geç

    # Hedef yolu belirleyip kopyalama
    dst = os.path.join(train_dir, label, fname)
    shutil.copyfile(src, dst)

In [13]:
# Doğrulama görüntülerini transfer etme
for image in val_list:
    fname = image + '.jpg'
    label = df.loc[image, 'dx']
    # Dosyanın hangi klasörde olduğunu kontrol etme
    if fname in folder_1:
        src = os.path.join(data_dir, 'ham10000_images_part_1', fname)
    elif fname in folder_2:
        src = os.path.join(data_dir, 'ham10000_images_part_2', fname)
    else:
        continue  # Dosya bulunamazsa geç

    # Hedef yolu belirleyip kopyalama
    dst = os.path.join(val_dir, label, fname)
    shutil.copyfile(src, dst)

In [14]:
# Her klasörde kaç görüntü olduğunu kontrol etme(original/+augmented)
for folder in [train_dir, val_dir]:
    print(folder)
    for name in class_labels:
       list = os.listdir(folder+'/'+name) # dir = directory path
       print(len(list))

C:/Users/esraa/Desktop/ham10000/base_dir\train_dir
294
463
989
103
1002
6034
128
C:/Users/esraa/Desktop/ham10000/base_dir\val_dir
33
51
110
12
111
671
14


In [15]:
# Veriyi genişletme
# 'nv' sınıfı genişletilmeyecek
class_list = ['mel', 'bkl', 'bcc', 'akiec', 'vasc', 'df']

for item in class_list:
    # Genişletilmiş görüntüler için geçici bir dizin oluşturma
    aug_dir = 'aug_dir'
    os.mkdir(aug_dir)
    # Aynı sınıf görüntülerini base_dir'a kopyalamak için dizin oluşturma
    img_dir = os.path.join(aug_dir,'img_dir')
    os.mkdir(img_dir)

    # Sınıf seçme
    img_class = item
    # Dizindeki tüm görüntüleri listeleme
    img_list = os.listdir('base_dir/train_dir/' + img_class)

    # Sınıf train_dır 'dan img_dir 'a görüntüleri kopyala
    for fname in img_list:
        # görüntünün kaynak yolu
        src = os.path.join('base_dir/train_dir/' + img_class, fname)
        # görüntünün hedef yolu
        dst = os.path.join(img_dir, fname)
        # kaynaktan hedefe görüntüyü kopyalama
        shutil.copyfile(src, dst)

    # Görüntülerin kendilerini değil, görüntüleri içeren dır 'ı işaret etme
    path = aug_dir
    save_path = 'base_dir/train_dir/' + img_class

    # Görüntüleri gerçek zamanlı olarak artırma
    datagen = ImageDataGenerator(rescale=1./255,
                                 rotation_range=10,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True,
                                 shear_range=0.2,
                                 channel_shift_range=10,
                                 fill_mode='nearest')

    batch_size = 128
    aug_datagen = datagen.flow_from_directory(path,
                                              save_to_dir=save_path,
                                              save_format='jpg',
                                              target_size=(224, 224),
                                              batch_size=batch_size)

    # Artırılmış görüntüleri oluşturma ve bunları eğitim klasörlerine ekleme
    # Her sınıfta sahip olmak istenen toplam görüntü sayısı
    num_aug_wanted = 6000
    num_files   = len(os.listdir(img_dir))
    num_batches = int( np.ceil((num_aug_wanted - num_files) / batch_size) )

    # Generator'a çalıştırma ve yaklaşık 6000 artırılmış görüntü oluşturma
    for i in range(0, num_batches):
        imgs, labels = next(aug_datagen)

    # Ham görüntü dosyalarıyla beraber geçici dizini silme
    shutil.rmtree('aug_dir')


Found 1002 images belonging to 1 classes.
Found 989 images belonging to 1 classes.
Found 463 images belonging to 1 classes.
Found 294 images belonging to 1 classes.
Found 128 images belonging to 1 classes.
Found 103 images belonging to 1 classes.


In [16]:
# Her klasörde kaç görüntü olduğunu kontrol etme(original/+augmented)
for folder in [train_dir, val_dir]:
    print(folder)
    for name in class_labels:
       list = os.listdir(folder+'/'+name) # dir = directory path
       print(len(list))

C:/Users/esraa/Desktop/ham10000/base_dir\train_dir
4704
5556
5934
4944
6012
6034
6016
C:/Users/esraa/Desktop/ham10000/base_dir\val_dir
33
51
110
12
111
671
14
